# Dependencies

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.odr import ODR, Model, Data, RealData
from scipy.optimize import curve_fit
import seaborn as sns
import uncertainties as uc
from uncertainties import ufloat
from uncertainties import unumpy as unp
from uncertainties.umath import sqrt
from scipy.optimize import fsolve
from scipy.interpolate import interp1d
from scipy.signal import correlate
from scipy.signal import find_peaks
from scipy.signal import savgol_filter
from scipy.fft import fft
from scipy.optimize import curve_fit
from scipy.stats import linregress

# Formatting

In [2]:
plt.style.use('Solarize_Light2')   #Options
                                    #Solarize_Light2
                                    #_classic_test_patch
                                    #_mpl-gallery
                                    #_mpl-gallery-nogrid
                                    #bmh
                                    #classic
                                    #dark_background
                                    #fast
                                    #fivethirtyeight
                                    #ggplot
                                    #grayscale
                                    #seaborn-v0_8
                                    #seaborn-v0_8-bright
                                    #seaborn-v0_8-colorblind
                                    #seaborn-v0_8-dark
                                    #seaborn-v0_8-dark-palette
                                    #seaborn-v0_8-darkgrid
                                    #seaborn-v0_8-deep
                                    #seaborn-v0_8-muted
                                    #seaborn-v0_8-notebook
                                    #seaborn-v0_8-paper
                                    #seaborn-v0_8-pastel
                                    #seaborn-v0_8-poster
                                    #seaborn-v0_8-talk
                                    #seaborn-v0_8-ticks
                                    #seaborn-v0_8-white
                                    #seaborn-v0_8-whitegrid
                                    #tableau-colorblind10

plt.rcParams.update({

    'figure.dpi': 225,                 # Resolution of figures
    'font.size': 10,                   # Default font size
    'font.family': 'sans-serif',       # Font family
    'font.sans-serif': ['DejaVu Sans'],
    
    'axes.titlesize': 10,              # Title font size
    'axes.labelsize': 8,              # Axis label font size
    'axes.linewidth': 0.8,             # Width of the axis lines
    'axes.grid': True,                 # Show grid
    # 'grid.color': '#dddddd',
    'grid.linestyle': '--',            # Grid line style (dashed)
    'grid.linewidth': 0.5,             # Grid line width
    
    'xtick.labelsize': 8,             # X-axis tick label size
    'ytick.labelsize': 8,             # Y-axis tick label size
    'xtick.direction': 'in',           # X-axis tick direction
    'ytick.direction': 'in',           # Y-axis tick direction

    'lines.linewidth': 0.8,            # Line width for plots
     'lines.markersize': 0.3,             # Marker size for scatter plots
     'lines.marker': 'o',               # Default marker shape

    'legend.frameon': True,            # Frame around legend
    'legend.framealpha': 0.8,          # Transparency of legend frame
    'legend.fontsize': 9,             # Font size in legend
    'legend.loc': 'best',              # Default legend location
})


# Useful Functions

In [3]:
def rSquared(f_model, x, y, Params):
    y_pred = f_model(*Params, x)  
    ss_res = np.sum((y - y_pred) ** 2)  
    ss_tot = np.sum((y - np.mean(y)) ** 2)  
    return 1 - (ss_res / ss_tot)

def Residuals(y_measured,y_modelled):
    return y_measured - y_modelled

def MeanSquareError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def RootMeanSquareError(y_measured,y_modelled):
    return np.sqrt(MeanSquareError(y_measured,y_modelled))

def MeanAbsError(y_measured,y_modelled):
    return ((1/len(y_measured))*(np.sum(np.abs(y_measured-y_modelled))))

def ConfidenceInterval(y_measured,y_modelled,confInt=95):
    if(confInt == 95):
        Z = 1.96
    elif(confInt == 90):
        Z = 1.645
    elif(confInt == 99):
        Z = 2.576
    return Z*np.sqrt((1/len(y_measured))*(np.sum((y_measured-y_modelled)**2)))

def ArrayWithStaticUnc(Array,unc):
    a = []
    b = []
    for i in range(len(Array)):
            a.append(uc.ufloat(Array[i],unc))
            b.append(unc)
    return np.array(a), np.array(b)

def UnpackUncArray(Array):
     value = [Array[i].nominal_value for i in range(len(Array))]
     std = [Array[i].std_dev for i in range(len(Array))]
     return np.array(value),np.array(std) 

# Non Linear Curve Fiting Function
def NonLinAnalysis(f_model, x, y, ParamsArr, weights):
    if ParamsArr is None:
        Params = None
        std = None 
        R2 = rSquared(f_model,x,y,Params)
    else:
        Params, cov = curve_fit(f_model, x, y, p0=ParamsArr, sigma=weights)
        std = np.sqrt(np.diag(cov))
        R2 = rSquared(f_model,x,y,Params)
    return Params, std, R2
 
def RR2(y,y_pred):
   ss_res = np.sum((y - y_pred) ** 2)
   ss_tot = np.sum((y - np.mean(y)) ** 2)
   R2 = 1 - (ss_res / ss_tot)
   return R2

def sortByArray1(array1, array2):
    sorted_pairs = sorted(zip(array1, array2))
    sorted_array1, sorted_array2 = zip(*sorted_pairs)
    return list(sorted_array1), list(sorted_array2)

# Inputting and Formatting Data

## Uncertainty Functions

In [4]:
def seriesresistors(*resistors):
    return sum(resistors)

def parallelresistors(*resistors):
    inverseRt = sum(1 / r for r in resistors)
    return 1/inverseRt


def linear_model(x, a, b):
    return a*x + b

def U_theoretical(RL, U, R1, R2):
    Rp = RL*R2/(R2+RL)
    dem = R1 + Rp
    return U/dem

def y_task2(RL, R1, R2):
    Rp = RL*R2/(RL + R2)
    a = R1 + Rp
    return a/Rp

In [5]:
def T3_R4_Unc(R4): 
    return 0.01*R4 #1% uncertainty of variable resistor
def Unc200mV(U): 
    return 0.005*U + 3*(0.01e-3) #±(0.5% + 3 Digit) Digit=0.01mV
def Unc2Vto20V(U):
    return 0.005*U + 3*0.01 #±(0.5% + 3 Digit) Digit=0.01V
def Unc200Ohm(R):
    return 0.008*R + 5*0.1#±(0.8% + 5 Digit) Digit=0.1 Ohm
def Unc2000Ohm(R):
    return 0.008*R + 3*0.1#±(0.8% + 2 Digit) Digit=0.1 Ohm COUILD BE 1 OHM!!!!!! CHECK NOTEBOOK
def Unc2mAto20mA(I):
    return 0.008*I + 3*0.01 #±(0.8% + 3 Digit) Digit = 0.01mA

In [6]:
Task1 = pd.read_csv('Data/Task1.csv', delimiter=',')
brr = [uc.ufloat(val, Unc200Ohm(val)) for val in Task1['R']]
R = np.array(brr) #Ohm
brr1 = [uc.ufloat(val, Unc2mAto20mA(val)) for val in Task1['I']]
I = np.array(brr1) #A
brr2 = [uc.ufloat(val, Unc200mV(val)) for val in Task1['U']]
U = np.array(brr2) #V
brr3 = [uc.ufloat(val, Unc200mV(val)) for val in Task1['U0']]
T1_Ut = np.array(brr3) #V

Task2 = pd.read_csv('Data/Task2.csv', delimiter = '\t' )
meow = [uc.ufloat(val, Unc200mV(val)) for val in Task2['U']]
T2_Ut = np.array(meow) #V
miau = [uc.ufloat(val, Unc200mV(val)) for val in Task2['UL']]
UL = np.array(miau)#V
miaow = [uc.ufloat(val, Unc2000Ohm(val)) for val in Task2['RL']] 
RL = np.array(miaow)#Ohm

In [7]:
Task3 = pd.read_csv("Data/Task3.csv")
Task3 = Task3.loc[Task3.index != 10].reset_index(drop=True)
Task3["R4(Ohms)mitUnc"] = [uc.ufloat(i,T3_R4_Unc(abs(i))) for i in Task3["R4(Ohms)"]]
Task3["R4(Ohms)nurUnc"] = UnpackUncArray(Task3["R4(Ohms)mitUnc"])[1]
Task3["U(Volts)mitUnc"] = [uc.ufloat(i,Unc2Vto20V(i)) for i in Task3["U(Volts)"]]
Task3["U(Volts)nurUnc"] = UnpackUncArray(Task3["U(Volts)mitUnc"])[1]
Task3["UB(Volts)mitUnc"] = np.concatenate(
    [np.array([uc.ufloat(i, Unc2Vto20V(abs(i))) for i in Task3.loc[0:6, "UB(Volts)"]]),
     np.array([uc.ufloat(i, Unc200mV(abs(i))) for i in Task3.loc[7:, "UB(Volts)"]])])
Task3["UB(Volts)nurUnc"] = UnpackUncArray(Task3["UB(Volts)mitUnc"])[1]

R1_T3 = uc.ufloat(100.2,Unc200Ohm(100.2))
R2_T3 = uc.ufloat(997,Unc2000Ohm(997))
R3_T3 = R1_T3




Task4 = pd.read_csv("Data/Task4.csv")
Task4["Ua/Ue"] = Task4["Ua(A)(mV)"]*1e-3 / Task4["Ue(B)(V)"]
Task4["f(Hz)mitUnc"] = [uc.ufloat(f,0.2) for f in Task4["f(Hz)"]]
R_T4 = uc.ufloat(997,Unc2000Ohm(997))
C_T4 = uc.ufloat(5.81e-9,(5.81e-9*0.025 + 20*(0.01e-9)))  #±(2.5% + 20 Digit) Digit = 0.01nF
Fc_T4_measured = 27475 #Hz
Fc_calc = (1/(2*np.pi*R_T4*C_T4))

# Task I

In [ ]:
print(U[0:3], R[0:3])

In [ ]:
#series
R_series = [x.nominal_value for x in R[:3]]
R_error = [x.std_dev for x in R[:3]]

U_series = [x.nominal_value for x in U[:3]]
U_error = [x.std_dev for x in U[:3]]

I_series = [x.nominal_value for x in I[:3]]
I_error = [x.std_dev for x in I[:3]]

U_th = [i * r for i, r in zip(I_series, R_series)]
U_therr = [i * r for i, r in zip(I_error, R_error)]

slope, intercept, r_value, p_value, std_err = linregress(R_series, U_series)
U_pred = [slope * r + intercept for r in R_series]

#plt.scatter(R_series, U_series, label='Experimental Data', color='blue', s=5) 
#plt.fill_between(R_series, 
#                 [u - e for u, e in zip(U_series, U_error)], 
#                 [u + e for u, e in zip(U_series, U_error)], 
#                 color='lightblue', alpha=0.2, label='Experimental Error') 
plt.errorbar(
    R_series, 
    U_series, 
    yerr=U_error, 
    xerr=None, 
    fmt='o',
    markersize=2,
    color='blue',
    label='Experimental Data', 
    capsize=1,
    ecolor="k"
)
plt.plot(R_series, U_th, label='Theoretical Data', color='red', linestyle="--")  
plt.fill_between(R_series, 
                 [u - e for u, e in zip(U_th, U_therr)], 
                 [u + e for u, e in zip(U_th, U_therr)], 
                 color='lightcoral', alpha=0.2, label='Theoretical Error')

print('therror: ', I_error, R_error, U_therr)
plt.plot(R_series, U_pred , label='Best Fit Line', color='purple')

print('The slope is: ', slope)
print('The intercept is: ', intercept)

R2s = RR2(np.array(U_series), np.array(U_pred))
print('R2 is: ', R2s)
Table1 = [
        [r"$r^2$", f"{R2s:.5g}"],
        ]
plt.table(cellText= Table1,
                  colLabels=["","Modelled"],
                  loc='lower center',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)

plt.xlabel('R(Ω)')
plt.ylabel('U(V)')
plt.title('Series Circuit')
plt.legend()
plt.grid()
plt.show()

#parallel 
R_parallel = [x.nominal_value for x in R[3:6]]
R_errorp = [x.std_dev for x in R[3:6]]

U_parallel = [x.nominal_value for x in U[3:6]]
U_errorp = [x.std_dev for x in U[3:6]]

I_parallel = [x.nominal_value for x in I[3:6]]
I_errorp = [x.std_dev for x in I[3:6]]

U_thp = [i * r for i, r in zip(I_parallel, [x.nominal_value for x in R[3:6]])]
U_therrp = [ i * r for i, r in zip(I_errorp, R_errorp)]

#plt.scatter(R_parallel, U_parallel, marker='o', label='Experimental Data', color='blue', s=5) 
#plt.fill_between(R_parallel, 
#                 [u - e for u, e in zip(U_parallel, U_errorp)], 
#                 [u + e for u, e in zip(U_parallel, U_errorp)], 
#                 color='lightblue', alpha=0.2, label='Experimental Error')  

plt.errorbar(
    R_parallel, 
    U_parallel, 
    yerr=U_errorp, 
    xerr=None, 
    fmt='o',
    markersize=2,
    color='blue',
    label='Experimental Data', 
    capsize=1,
    ecolor="k"
)

plt.plot(R_parallel, U_thp, marker='o', label='Theoretical Data', color='red', linestyle="--")
plt.fill_between(R_parallel, 
                 [u - e for u, e in zip(U_thp, U_therrp)], 
                 [u + e for u, e in zip(U_thp, U_therrp)], 
                 color='lightcoral', alpha=0.2, label='Theoretical Error')

slopep, interceptp, r_value, p_valuep, std_errp = linregress(R_parallel, U_parallel)
U_predp = [slopep * r + interceptp for r in R_parallel]
plt.plot(R_parallel, U_predp, label='Best Fit Line', color='purple')

print('The slope is: ', slopep)
print('The intercept is: ', interceptp)
R2p = RR2(np.array(U_parallel), np.array(U_predp))
print('R2 is: ', R2p)
Table2 = [
        [r"$r^2$", f"{R2p:.5g}"],
        ]
plt.table(cellText= Table2,
                  colLabels=["","Modelled"],
                  loc='lower center',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)

plt.xlabel('R(Ω)')
plt.ylabel('U(V)')
plt.title('Parallel Circuit')
plt.legend()
plt.grid()
plt.show()

In [ ]:
R_parallel_vals = [ufloat(100.2, 1.3016), ufloat(199.9, 2.0992), ufloat(997.0, 8.476)]
U_parallel_vals = [ufloat(3.85, 0.01718), ufloat(3.91, 0.01958), ufloat(4.35, 0.02178)]
I_parallel_vals = [ufloat(0.03473, 0.03029), ufloat(0.0177, 0.03014), ufloat(0.00402, 0.03003)]

R_parallel_vals_nominal = [x.nominal_value for x in R_parallel_vals]  # Extract nominal values
U_parallel_vals_nominal = [x.nominal_value for x in U_parallel_vals]

R_parallel_errors = [x.std_dev for x in R_parallel_vals]  # Extract uncertainties
U_parallel_errors = [x.std_dev for x in U_parallel_vals]
I_parallel_errors = [x.std_dev for x in I_parallel_vals]

U_theoretical_parallel = [i.nominal_value * r.nominal_value for i, r in zip(I_parallel_vals, R_parallel_vals)]
U_theoretical_errors_parallel = [unp.nominal_values(i) * unp.std_devs(r) for i, r in zip(I_parallel_vals, R_parallel_vals)]



#plt.scatter(R_parallel_vals_nominal, U_parallel_vals_nominal, marker='o', label='Experimental Data', color='blue')  
#plt.fill_between(R_parallel_vals_nominal, 
#                 [u - e for u, e in zip(U_parallel_vals_nominal, U_parallel_errors)], 
#                 [u + e for u, e in zip(U_parallel_vals_nominal, U_parallel_errors)], 
#                 color='green', alpha=0.2, label='Experimental Error')
plt.errorbar(
    R_parallel_vals_nominal, 
    U_parallel_vals_nominal, 
    yerr=U_parallel_errors, 
    xerr=None, 
    fmt='o',
    markersize=2,
    color='blue',
    label='Experimental Data', 
    capsize=1,
    ecolor="k"
)
plt.plot(R_parallel_vals_nominal, U_theoretical_parallel, marker='o', label='Theoretical Data', color='purple')
plt.fill_between(R_parallel_vals_nominal, 
                 [u - e for u, e in zip(U_theoretical_parallel, U_theoretical_errors_parallel)], 
                 [u + e for u, e in zip(U_theoretical_parallel, U_theoretical_errors_parallel)], 
                 color='lightcoral', alpha=0.2, label='Theoretical Error')

slope_parallel, intercept_parallel, r_value_parallel, p_value_parallel, std_err_parallel = linregress(R_parallel_vals_nominal, U_parallel_vals_nominal)
U_predicted_parallel = [slope_parallel * r + intercept_parallel for r in R_parallel_vals_nominal]
plt.plot(R_parallel_vals_nominal, U_predicted_parallel, label='Best Fit Line', color='red')

print('The slope is: ', slope_parallel)
print('The intercept is: ', intercept_parallel)
R2_parallel = RR2(np.array(U_parallel_vals_nominal), np.array(U_predicted_parallel))
print('R2 is: ', R2_parallel)
Table2 = [
    [r"$r^2$", f"{R2_parallel:.5g}"],
]
plt.table(cellText=Table2,
           colLabels=["", "Modelled"],
           loc='lower center',
           cellLoc='left',
           colWidths=[0.1, 0.12],
           fontsize=4, zorder=100)

plt.xlabel('R(Ω)')
plt.ylabel('U(V)')
plt.title('Parallel Circuit')
plt.legend()
plt.grid()
plt.show()

# Task II

In [ ]:
x_exp = 1 / np.array([rl.nominal_value for rl in RL], dtype=float)
y_exp_nominal = np.array([x.nominal_value for x in (T2_Ut / UL)], dtype=float)
y_exp_uncertainties = np.array([x.std_dev for x in (T2_Ut / UL)], dtype=float)

y_th_nominal = np.array([y.nominal_value for y in y_task2(RL, 100.2, 997)], dtype=float)
y_th_uncertainty = np.array([y.std_dev for y in y_task2(RL, 100.2, 997)], dtype=float)


plt.figure(figsize=(8, 6))
plt.plot(x_exp, y_th_nominal, label='Theoretical Data', color='red', linestyle="--")
plt.fill_between(x_exp, y_th_nominal - y_th_uncertainty, y_th_nominal + y_th_uncertainty, alpha=0.2, color='lightcoral', label='Theoretical Error')


#plt.scatter(x_exp, y_exp_nominal, label='Experimental Data', color='blue', marker='o', s = 5)
#plt.fill_between(x_exp, y_exp_nominal - y_exp_uncertainties, y_exp_nominal + y_exp_uncertainties, alpha=0.2, color='lightblue', label='Experimental Error')
plt.errorbar(
    x_exp, 
    y_exp_nominal, 
    yerr=y_exp_uncertainties, 
    xerr=None, 
    fmt='o',
    markersize=2,
    color='blue',
    label='Experimental Data', 
    capsize=1,
    ecolor="k"
)

plt.xlabel("1/RL(1/Ω)")
plt.ylabel("U/UL")
plt.title("Voltage Divider Under Load")
plt.legend()
plt.grid()
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sl, inter, r_val, p_val, std_er = linregress(x_exp, y_exp_nominal)
y_pred = [sl * r + inter for r in x_exp]
#plt.scatter(x_exp, y_exp_nominal, label='Experimental Data', color='blue', marker='o', s=5)
#plt.fill_between(x_exp, y_exp_nominal - y_exp_uncertainties, y_exp_nominal + y_exp_uncertainties, alpha=0.2, color='lightblue', label='Experimental Error')
plt.errorbar(
    x_exp, 
    y_exp_nominal, 
    yerr=y_exp_uncertainties, 
    xerr=None, 
    fmt='o',
    markersize=2,
    color='blue',
    label='Experimental Data', 
    capsize=1,
    ecolor="k"
)

plt.plot(x_exp, y_pred , label='Best Fit Line', color='red')

R2l = RR2(y_exp_nominal, y_pred)
print('R2 is: ', R2l)
Table3 = [
        [r"$r^2$", f"{R2l:.5g}"],
        ]
plt.table(cellText= Table3,
                  colLabels=["","Modelled"],
                  loc='lower center',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)
plt.xlabel("1/RL(1/Ω)")
plt.ylabel("U/UL")
plt.title("Voltage Divider Under Load")
plt.legend()
plt.grid()
plt.show()

print('slope: ', sl)
print('intercept: ', inter)

# Task III

### First Page
- [x] Set up a wheatstone bridge. 
- [x] Measure the bridge voltage as a function of the resistance under load.
- [ ] Plot the data and compare with predictions

### During your Preparation
- [ ] Calculate the bridge voltage $U_b$ as a function of the applied voltage $U$ and the resistance $R_1$, $R_2$, $R_3$ and $R_4$.
- [ ] Consider the case $R_1 = R_3$ and $R_2 = R_4$ what can this setup be used for?
### In the Experiment
- [x] Choose resistances $R_1 = R_3$ and use a variable resistor in place of $R_4$
- [x] Measure the bridge voltage $U_b$ as a function of $R_4$.
- [x] Plot the $U_b$ versus $R_4$ and determine $R_2$

### Checklist
- [ ] Two Graphs:
    - [x] Theory and Practice in comparison
    - [ ] For evaluation with nonlinear fit
- [ ] Calculations of Errors

### Extra Notes
- [ ] Tables with Measurements

In [ ]:
Task3

In [ ]:
def WheatstoneBridgeVoltage(R4,R1,R2):
    return Task3["U(Volts)mitUnc"]*((R2/(R2+R1))-(R4/(R4+R1)))
def WheatstoneBridgeVoltageNoUnc(R4,R1,R2):
    return Task3["U(Volts)"]*((R2/(R2+R1))-(R4/(R4+R1)))
WBV_NominalValues, WBV_StdDev = np.array(UnpackUncArray(WheatstoneBridgeVoltage(Task3["R4(Ohms)mitUnc"],R1_T3,R2_T3)))

# plt.scatter(Task3["R4(Ohms)"], Task3["UB(Volts)"],s=2, color="#A15E49", label="Experimental")
# plt.errorbar(Task3["R4(Ohms)"],Task3["UB(Volts)"],yerr=Task3["UB(Volts)nurUnc"],xerr=Task3["R4(Ohms)nurUnc"], color="#A15E49", label="Experimental",capsize=1,)
plt.errorbar(
    Task3["R4(Ohms)"], 
    Task3["UB(Volts)"], 
    yerr=Task3["UB(Volts)nurUnc"], 
    xerr=Task3["R4(Ohms)nurUnc"], 
    fmt='o',
    markersize=2,
    color="#FF5733",
    label="Experimental", 
    capsize=1,
    ecolor="k"
)

plt.ylabel(r"$U_b$ (V)")
plt.xlabel(r"$R_4$ (Ω)")
plt.title("Wheatstone Bridge Voltage")
plt.plot(Task3["R4(Ohms)"],WBV_NominalValues,color="#1F618D",label="Theoretical", linestyle="--")
plt.fill_between(Task3["R4(Ohms)"],WBV_NominalValues-WBV_StdDev,WBV_NominalValues+WBV_StdDev,color="#85C1AE",alpha=0.5,label="Theoretical Uncertainty")
WBRR1 = RR2(Task3["UB(Volts)"],WBV_NominalValues)
WB_Table1 = [
        [r"$r^2$", f"{WBRR1:.5g}"],
        ]
plt.table(cellText=WB_Table1,
                  colLabels=["","Theory"],
                  loc='center',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)
plt.legend()
plt.show()

In [ ]:
plt.errorbar(
    Task3["R4(Ohms)"], 
    Task3["UB(Volts)"], 
    yerr=Task3["UB(Volts)nurUnc"], 
    xerr=Task3["R4(Ohms)nurUnc"], 
    fmt='o',
    markersize=2,
    color="#FF5733",
    label="Experimental", 
    capsize=1,
    ecolor="k"
)
plt.ylabel(r"$U_b$ (V)")
plt.xlabel(r"$R_4$ (Ω)")
WBParams, WBstd, WBRR1 = NonLinAnalysis(WheatstoneBridgeVoltageNoUnc,Task3["R4(Ohms)"],Task3["UB(Volts)"],[R1_T3.nominal_value,R2_T3.nominal_value], None)
WBModel_NominalValues, WBModel_StdDev = UnpackUncArray(WheatstoneBridgeVoltage(Task3["R4(Ohms)mitUnc"],uc.ufloat(WBParams[0],WBstd[0]),uc.ufloat(WBParams[1],WBstd[1])))
WBRR1 = RR2(Task3["UB(Volts)"],WBModel_NominalValues)
plt.plot(Task3["R4(Ohms)"],WBModel_NominalValues,color="#0B3948",label="Fitting",linestyle="--")
plt.fill_between(Task3["R4(Ohms)"],WBModel_NominalValues-WBModel_StdDev,WBModel_NominalValues+WBModel_StdDev,color="#85C1AE",alpha=0.5,label="Fitting Uncertainty")
WB_Table2 = [
        [r"$r^2$", f"{WBRR1:.5g}"],
        [r"$R_1 = R_3$", f"{WBParams[0]:.3g}± {WBstd[0]:.3g}Ω"],
        [r"$R_2$", f"{WBParams[1]:.3g}±{WBstd[1]:.3g} Ω"],
        # [r"$R_3$",f"{WBParams[0]:.3g}± Ω"],
        ]
plt.table(cellText=WB_Table2,
                  colLabels=["","Modelled"],
                  loc='center',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)
plt.title("Wheatstone Bridge Voltage Fit Analysis")
plt.legend()
plt.show()

# Task IV

In [ ]:
def HighPassFilter(x,R,C):
    """
    This code uses complex numbers 
    """
    result = []
    for f in x:
        # K = (0,b)±(0,Bunc)
        b = (f/(1/(2*np.pi*R*C))).nominal_value
        Bunc = (f/(1/(2*np.pi*R*C))).std_dev
        Result_magnitude = np.sqrt((b**2/(1+b**2))**2 + (b/(1+b**2))**2)
        KaUnc = (((b**2/(1+b**2)) * np.abs((((2*b)/(b**2+1)**2)*Bunc)))/Result_magnitude)
        KbUnc = (((b/(1+b**2))* np.abs(((1-b**2)/(b**2+1)**2)*Bunc))/Result_magnitude)
        Result_uncertainty = np.sqrt(KaUnc**2 + KbUnc**2 )
        result.append(uc.ufloat(Result_magnitude,Result_uncertainty))
    return result
plt.errorbar(
    Task4["f(Hz)"], 
    Task4["Ua/Ue"], 
    yerr=np.ones(len(Task4["f(Hz)"]))*(7.8125e-3), 
    xerr=np.ones(len(Task4["f(Hz)"]))*0.2, 
    fmt='o',
    markersize=2,
    color="#D94A38",
    label="Experimental", 
    capsize=1,
    ecolor="k"
)
HPF_NominalValues, HPF_StdDev = np.array(UnpackUncArray(HighPassFilter(Task4["f(Hz)mitUnc"],R_T4,C_T4)))
HPFR2 = RR2(Task4["Ua/Ue"],HPF_NominalValues)
plt.plot(Task4["f(Hz)"],HPF_NominalValues,color="#0B3948",label="Fitting",linestyle="--")
plt.fill_between(Task4["f(Hz)"],HPF_NominalValues-HPF_StdDev,HPF_NominalValues+HPF_StdDev,color="#85C1AE",alpha=0.5,label="Fitting Uncertainty")
plt.axvline(Fc_T4_measured,color="gray",linestyle="--",label=r"$f_c$")
HPF_Table1 = [
        [r"$r^2$", f"{HPFR2:.5g}"],
        ]
plt.table(cellText=HPF_Table1,
                  colLabels=["","Theory"],
                  loc='lower right',
                  cellLoc='left',
                  colWidths=[0.1,0.12]
                  ,fontsize=4,zorder=100)
plt.xlabel(r"$f$ (Hz)")
plt.xscale('log')
plt.yscale("log")
plt.ylabel(r"$\frac{U_a}{U_e}$")
plt.title("High Pass Filter Results")
plt.legend()
plt.show()

In [17]:
def HighPassFilter(x,R,C):
    """
    This code uses complex numbers 
    """
    result = []
    for f in x:
        # K = (0,b)±(0,Bunc)
        b = (f/(1/(2*np.pi*R*C))).nominal_value
        Bunc = (f/(1/(2*np.pi*R*C))).std_dev
        Result_magnitude = np.sqrt((b**2/(1+b**2))**2 + (b/(1+b**2))**2)
        KaUnc = (((b**2/(1+b**2)) * np.abs((((2*b)/(b**2+1)**2)*Bunc)))/Result_magnitude)
        KbUnc = (((b/(1+b**2))* np.abs(((1-b**2)/(b**2+1)**2)*Bunc))/Result_magnitude)
        Result_uncertainty = np.sqrt(KaUnc**2 + KbUnc**2 )
        result.append(uc.ufloat(Result_magnitude,Result_uncertainty))
    return result

def HighPassFilterNoUFloat(x,R,C):
    """
    This code uses complex numbers 
    """
    result = []
    for f in x:
        # K = (0,b)±(0,Bunc)
        b = (f/(1/(2*np.pi*R*C)))
        Bunc = (f/(1/(2*np.pi*R*C)))
        Result_magnitude = np.sqrt((b**2/(1+b**2))**2 + (b/(1+b**2))**2)
        result.append(Result_magnitude)
    return result

In [ ]:
initial_guess = [R_T4.nominal_value, C_T4.nominal_value]

popt, pcov = curve_fit(HighPassFilterNoUFloat, Task4["f(Hz)"], Task4["Ua/Ue"], p0=initial_guess,sigma=(7.8125e-3))
fitted_curve = HighPassFilterNoUFloat(Task4["f(Hz)"], *popt)
HPFRR22 = RR2(Task4["Ua/Ue"],fitted_curve)
R_fit, C_fit = popt
pstd = np.sqrt(np.diag(pcov))

plt.errorbar(
    Task4["f(Hz)"], 
    Task4["Ua/Ue"], 
    yerr=np.ones(len(Task4["f(Hz)"]))*(7.8125e-3), 
    xerr=np.ones(len(Task4["f(Hz)"]))*0.2, 
    fmt='o',
    markersize=2,
    color="#D94A38",
    label="Experimental", 
    capsize=1,
    ecolor="k"
)
plt.plot(Task4["f(Hz)"], fitted_curve, linestyle='--',color="#0B3948",label="Fitting")
HPF_Table2 = [
        [r"$r^2$", f"{HPFRR22:.5g}"],
        [r"$R$", f"({popt[0]:.5g}±{pstd[0]:.3g})Ω"],
        [r"$C$", f"({popt[1]:.5g}±{pstd[1]:.3g})F"],
        # [r"$R_3$",f"{WBParams[0]:.3g}± Ω"],
        ]
plt.table(cellText=HPF_Table2,
                  colLabels=["","Modelled"],
                  loc='lower right',
                  cellLoc='left',
                  colWidths=[0.1,0.3]
                  ,fontsize=4,zorder=100)
plt.axvline(Fc_T4_measured,color="gray",linestyle="--",label=r"$f_c$")
plt.xlabel(r"$f$ (Hz)")
plt.ylabel(r"$\frac{U_a}{U_e}$")
plt.xscale("log")
plt.yscale("log")
plt.title("High Pass Filter Results")
plt.legend()
plt.show()